In [1]:
using Pkg
Pkg.activate("..")
using ElementalFunctionsTests

  Activating project at `~/Documents/Academia/taylor/ElementalFunctionsTests`
[ Info: Precompiling ElementalFunctionsTests [d4cb4dc5-4b71-4e54-85a0-20ff967ef3c3]


In [2]:
distance2inf(prevfloat(Inf))

LoadError: could not load library "./clibs/checklib"
dlopen(./clibs/checklib.dylib, 0x0001): tried: './clibs/checklib.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS./clibs/checklib.dylib' (no such file), '/Applications/Julia-1.9.app/Contents/Resources/julia/lib/julia/./clibs/checklib.dylib' (no such file), '/Applications/Julia-1.9.app/Contents/Resources/julia/lib/julia/.././clibs/checklib.dylib' (no such file), '/Applications/Julia-1.9.app/Contents/Resources/julia/lib/./clibs/checklib.dylib' (no such file), '/usr/lib/./clibs/checklib.dylib' (no such file, not in dyld cache), './clibs/checklib.dylib' (no such file)